In [1]:
import os
print(f"IMAGE_VERSION: {os.environ.get('IMAGE_VERSION', 'Not set')}")
print(f"SAGEMAKER_INTERNAL_IMAGE_URI: {os.environ.get('SAGEMAKER_INTERNAL_IMAGE_URI', 'Not set')}")

IMAGE_VERSION: 3.5.1-cpu
SAGEMAKER_INTERNAL_IMAGE_URI: Not set


In [2]:
from datetime import datetime
from zoneinfo import ZoneInfo
print(f"Execution Start Time (EST): {datetime.now(ZoneInfo('America/New_York')).strftime('%Y-%m-%d %H:%M:%S %Z')}")


Execution Start Time (EST): 2025-11-13 15:01:21 EST


In [3]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')


Using default sagemaker_studio package from environment


In [4]:
from sagemaker_studio import Project
proj = Project()
s3_shared_conn = proj.connection('default.s3_shared')
s3_bucket = s3_shared_conn.data.s3_uri.rstrip('/').split('/')[-2]
print(f"Using S3 bucket: {s3_bucket}")

Using S3 bucket: amazon-sagemaker-198737698272-us-east-2-4uoxu0uszmmxbd


# Test 05: Customer Churn Analysis using Spark Connect

This notebook performs customers churn analytics using spark connect

## 🎯 Objective

Demonstrate how Spark Connect can be used for end-to-end analytics:

* Building a pipeline that ingests and transforms customer data
* Performing feature engineering and aggregations
* Running analytical queries
* Visualizing results locally with matplotlib

In [5]:
print(f"Performing customer churn analysis using Spark version: {spark.version}")

NameError: name 'spark' is not defined

In [ ]:
import randomfrom pyspark.sql import Rowdef generate_customer(i):    age = random.randint(18, 75)    gender = random.choice(["Male", "Female"])    plan_type = random.choice(["Basic", "Standard", "Premium"])    tenure = random.randint(1, 72)    monthly_charges = round(random.uniform(20, 120), 2)    total_charges = round(monthly_charges * tenure, 2)    complaints = random.randint(0, 5)    churn = 1 if (tenure < 12 and monthly_charges > 80) or (complaints > 2) else 0    return Row(        customer_id=i,        age=age,        gender=gender,        plan_type=plan_type,        tenure=tenure,        monthly_charges=monthly_charges,        total_charges=total_charges,        complaints=complaints,        churn=churn    )data = [generate_customer(i) for i in range(5000)]df = spark.createDataFrame(data)df.show(5)

In [ ]:
df

In [ ]:
df.printSchema()df.describe().show()

In [ ]:
import matplotlib.pyplot as pltchurn_counts = df.groupBy("churn").count().toPandas()plt.bar(churn_counts["churn"], churn_counts["count"], color=["green", "red"])plt.title("Customer Churn Distribution")plt.xlabel("Churn")plt.ylabel("Count")plt.show()

In [ ]:
from pyspark.sql.connect import functions as Fdf_transformed = (    df.withColumn("avg_monthly_spend", F.col("total_charges") / F.col("tenure"))      .withColumn("is_high_value", F.when(F.col("monthly_charges") > 70, 1).otherwise(0))      .withColumn("complaint_rate", F.col("complaints") / F.col("tenure")))df_transformed.show(5)

In [ ]:
churn_by_plan = (    df_transformed.groupBy("plan_type", "is_high_value")    .agg(F.mean("churn").alias("churn_rate"))    .orderBy("churn_rate", ascending=False))churn_by_plan.show()

In [ ]:
churn_plot = churn_by_plan.toPandas()plt.figure(figsize=(6,4))for val in churn_plot['is_high_value'].unique():    subset = churn_plot[churn_plot['is_high_value'] == val]    plt.bar(subset['plan_type'], subset['churn_rate'], alpha=0.7, label=f"High Value={val}")plt.title("Churn Rate by Plan Type and Value Segment")plt.xlabel("Plan Type")plt.ylabel("Churn Rate")plt.legend()plt.show()

In [ ]:
high_churn_segments = (    df_transformed.filter(F.col("churn") == 1)    .groupBy("plan_type", "is_high_value")    .agg(F.count("*").alias("churned_customers"))    .orderBy("churned_customers", ascending=False))high_churn_segments.show(10)

In [ ]:
numeric_cols = ["age", "tenure", "monthly_charges", "total_charges", "complaints", "avg_monthly_spend"]corrs = [(c, float(df_transformed.stat.corr("churn", c))) for c in numeric_cols]spark.createDataFrame(corrs, ["feature", "correlation_with_churn"]).show()

### Key Insights
- Customers with shorter tenures and higher monthly charges tend to churn more.
- High-value customers on the Basic plan are at higher risk of churn.
- Complaints have a strong positive correlation with churn.
- Tenure and total charges have strong negative correlation with churn.